# *Cosmian Secure Computation:* Run any Python code on encrypted data

## Overview

On April 15, 1912, the Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone on board, resulting in the death of 1502 out of 2224 passengers and crew.

Alice owns the passenger data (name, age, gender, socio-economic class, etc.) and wants to perform statistical analysis with these sensitive data.

With *Cosmian Secure Computation*, she can do this analysis while their data stay encrypted the whole time.


## Step 1: Create computation (Computation Owner)

The first step is to create a computation and specify each role.

In this toy example, Alice, the `Computation Owner`, is also the `Code Provider`, the `Data Provider`, and the `Result Consumer`.

In a production setup, you can split these roles between different parties. Moreover, there can be several `Data Providers` and several `Result Consumers`.

To create a computation, Alice needs a secret token. They are available on the [Cosmian console](https://console.cosmian.com), on the [Secret token page](https://console.cosmian.com/secret-token). In this example, the tokens are stored in the `config.ini` file.

In [ ]:
# Import packages
!pip install cosmian_secure_computation_client --quiet --upgrade
!pip install pandas --quiet
!pip install seaborn --quiet
import pandas as pd
import seaborn as sns
from cosmian_secure_computation_client import ComputationOwnerAPI, CodeProviderAPI, DataProviderAPI, ResultConsumerAPI
from configparser import ConfigParser

# Get the Cosmian token (available on https://console.cosmian.com/secret-token)
config = ConfigParser()
config.read("config.ini")
TOKEN1: str = config["Credentials"]["token_1"]
computation_owner = ComputationOwnerAPI(TOKEN1)

# Create a new computation
computation = computation_owner.create_computation(
    'Titanic 🚢',
    code_provider_email="alice@cosmian.com",
    data_providers_emails=["alice@cosmian.com"],
    result_consumers_emails=["alice@cosmian.com"])

words = ComputationOwnerAPI.random_words()

---
## Step 2: Register (All participants)

To register, participants need a crypto context. To start a crypto context, all participants need to provide the shared words generated by the computation owner. 

Note: The `CryptoContext` will generate all the necessary cryptographic keys (an asymmetric key pair and a symmetric key), you can override these keys.
Note: If you split the the code in multiple files, you need to save and load the same crypto context every time.

In [ ]:
# Register Code Provider
computation_uuid = computation.uuid


from cosmian_secure_computation_client.crypto.context import CryptoContext

cp_crypto_context = CryptoContext(words=words)
code_provider = CodeProviderAPI(TOKEN1, cp_crypto_context)

computation = code_provider.register(computation_uuid)


# Register Data Provider 1
dp_crypto_context = CryptoContext(words=words)
data_provider1 = DataProviderAPI(TOKEN1, dp_crypto_context)

computation = data_provider1.register(computation_uuid)


# Register Result Consumer 1
rc_crypto_context = CryptoContext(words=words)
result_consumer1 = ResultConsumerAPI(TOKEN1, rc_crypto_context)

computation = result_consumer1.register(computation_uuid)

---
## Step 3: Provide code (Code Provider)

Alice provides the code she wrote. It must be organized like this:

```bash
code_titanic
├── secret_module.py
└── run.py
```

All the files in this folder (except `run.py`) are encrypted with the CryptoContext's symmetric key before being sent to the enclave.

Once the secure enclave receives the code, it is sealed and generates the enclave identity automatically.

In [ ]:
# Import package
from pathlib import Path
from cosmian_secure_computation_client.crypto.helper import random_symkey

# Upload the code, encrypted
path = Path("./code_titanic")
code_provider.upload(computation_uuid, path)

---
## Step 4: Wait the identity of the enclave

After all participants are registered and the code is uploaded the SGX enclave identity will be locked, and the enclave's keys will be generated.

In [ ]:
# Wait for the generation of the enclave identity, we are using the code provider object to do the waiting but 
# all API objects can do the waiting.
import time
from cosmian_secure_computation_client.api.computations import EnclaveIdentityLockError

while True:
    computation = code_provider.get_computation(computation_uuid)

    if computation.enclave.identity is None:
        print("Waiting 5s the generation of the enclave identity…")
        time.sleep(5)
    elif isinstance(computation.enclave.identity, EnclaveIdentityLockError):
        raise Exception(f"The enclave cannot lock its identity because there is an error in the entrypoint.\n\n{computation.enclave.identity.stdout}\n\n{computation.enclave.identity.stderr}")
    else:
        break

# Check the computation
quote = computation.enclave.identity.quote

# Print the quote
#print("\n\n")
#print(computation_owner.remote_attestation(quote))
#print("\n\n")

---
## Step 5: Approve computation (Code Provider)

Alice can now send her symmetric key, sealed with the public key of the enclave.

In [ ]:
# Send the sealed symmetric key
code_provider.key_provisioning(computation.uuid, computation.enclave.identity.public_key)

---
## Step 6: Send data (Data Providers)

As for the code, Alice push her data, encrypted with the CryptoContext's symmetric key before sending.

In [ ]:
# Load the dataset
titanic = sns.load_dataset('titanic')
titanic.to_csv("titanic.csv")

# Upload the data, encrypted
path_1 = Path("titanic.csv")
data_provider1.push_files(computation_uuid, [path_1])
data_provider1.done(computation_uuid)

Once the data are sent, Alice can check the enclave identity and send her symmetric key, sealed with the public key of the enclave.

In [ ]:
# Check the computation
computation = data_provider1.get_computation(computation_uuid)
quote = computation.enclave.identity.quote

# Print the quote
#print("\n\n")
#print(data_provider1.remote_attestation(quote))
#print("\n\n")

# Send the sealed symmetric key
data_provider1.key_provisioning(computation_uuid, computation.enclave.identity.public_key)

---
## Step 7: Run computation (Result Consumer)

Finally, Alice will send the symmetric key that she will use later to decrypt the result.

Again, this symmetric key is sealed with the public key of the enclave before being sent.

Once the enclave receives this sealed key, the computation starts automatically.

In [ ]:
# Check the computation
computation = result_consumer1.get_computation(computation_uuid);
quote = computation.enclave.identity.quote
#print("\n\n")
#print(result_consumer.remote_attestation(quote))
#print("\n\n")

# Send the sealed symmetric key
result_consumer1.key_provisioning(computation.uuid, computation.enclave.identity.public_key)

---
## Step 8: Get result (Result Consumer)

Once the run ends, the computation returns the exit code (`0` if everything worked, non-zero if there was an error) and the standard console outputs (`stdout` and `stderr`).

If the exit code is `0`, Alice can fetch her result which is decrypted with the CryptoContext's symmetric keys.

In [ ]:
# Wait for the end of the computation
while True:
    computation = result_consumer1.get_computation(computation_uuid)
    if computation.runs.current is None and len(computation.runs.previous) == 1:
        run = computation.runs.previous[0]
        if run.exit_code != 0:
            print("\n\n### Exit Code ###\n")
            print(run.exit_code)
            print("\n\n### stdout ###\n")
            print(run.stdout)
            print("\n\n### stderr ###\n")
            print(run.stderr)
            print("\n\n")
            raise "Run fail."
        else:
            break
    else:
        print("Waiting 10s end of computation…")
        time.sleep(10)

In [ ]:
# Download the result
result = result_consumer1.fetch_results(computation.uuid)

# Write the result
result_path: Path = Path("titanic_result.csv")
result_path.write_bytes(result)

# Plot the result
df = pd.read_csv(result_path)
df.rename(columns={'1':'Survival rate'}, inplace=True)
sns.catplot(x="class", y="Survival rate", col="who",data=df, kind="bar", palette="pastel")